In [1]:
# импортируем библиотеки pandas, numpy, math
import pandas as pd
import numpy as np
import math

In [2]:
# создаем собственный класс, выполняющий замену
# пропусков средним значением
class MeanImputer():
    """
    Параметры
    ---------
    copy: bool, по умолчанию True
         Возвращает копию.
   
    Возвращает
    -------
    X : pandas.DataFrame или numpy.ndarray
        Датафрейм pandas или массив NumPy
        с импутированными значениями.
    """
    
    def __init__(self, copy=True):
        # все параметры для инициализации публичных атрибутов 
        # должны быть заданы в методе __init__
        
        # публичный атрибут
        self.copy = copy
        
    def __is_numpy(self, X):
        # частный метод, который с помощью функции isinstance()
        # проверяет, является ли наш объект массивом NumPy
        return isinstance(X, np.ndarray)
                
    def fit(self, X, y=None):
        # метод .fit() должен принимать
        # в качестве аргументов X и y
        
        # создаем пустой словарь, в котором ключами
        # будут имена/целые числа, а значениями - средние
        self._encoder_dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат метода __is_numpy
        is_np = self.__is_numpy(X)
        
        # если объект - массив NumPy
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.nanmean(X[:, col])
        # если объект - датафрейм pandas
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = X[col].mean()
                
        # fit возвращает self
        return self
    
    def transform(self, X):
        # transform принимает в качестве 
        # аргумента только X
        
        # выполняем копирование массива во избежание 
        # предупреждения SettingWithCopyWarning
        # "A value is trying to be set on a copy of 
        # a slice from a DataFrame (Происходит попытка 
        # изменить значение в копии среза данных 
        # датафрейма)"
        if self.copy:
            X = X.copy()
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат метода __is_numpy
        is_np = self.__is_numpy(X)
        
        # применяем преобразование к X
        # если объект - массив NumPy:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # заменяем пропуски средним 
                # значением из словаря
                X[:, col] = np.nan_to_num(
                    X[:, col], 
                    nan=self._encoder_dict[col])
        # если объект - датафрейм pandas:
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # заменяем пропуски средним 
                # значением из словаря
                X[col] = np.where(X[col].isnull(), 
                                  self._encoder_dict[col], 
                                  X[col])
        
        # transform возвращает X
        return X

In [3]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]})
toy_train

,Balance,Age
0,8.3,23.0
1,NaN,29.0
2,10.2,36.0
3,3.1,NaN


In [4]:
# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame(
    {'Balance': [10.4, np.NaN, 22.5, 1.1], 
     'Age': [13, 19, 66, np.NaN]})
toy_test

,Balance,Age
0,10.4,13.0
1,NaN,19.0
2,22.5,66.0
3,1.1,NaN


In [5]:
# смотрим, как будут выглядеть преобразования в игрушечных
# обучающем и тестовом датафреймах pandas
for col in toy_train.columns:
    toy_train[col].fillna(toy_train[col].mean(), inplace=True)
    toy_test[col].fillna(toy_train[col].mean(), inplace=True)
print('обучающий датафрейм')
print(toy_train)
print('')
print('тестовый датафрейм')
print(toy_test)

обучающий датафрейм
   Balance        Age
0      8.3  23.000000
1      7.2  29.000000
2     10.2  36.000000
3      3.1  29.333333

тестовый датафрейм
   Balance        Age
0     10.4  13.000000
1      7.2  19.000000
2     22.5  66.000000
3      1.1  29.333333


In [6]:
# создаем экземпляр класса MeanImputer
imp = MeanImputer()
# обучаем модель
imp.fit(toy_train)
# выполняем преобразование игрушечного
# обучающего датафрейма pandas
toy_train = imp.transform(toy_train)
toy_train

,Balance,Age
0,8.3,23.000000
1,7.2,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [7]:
# выполняем преобразование игрушечного
# тестового датафрейма pandas
toy_test = imp.transform(toy_test)
toy_test

,Balance,Age
0,10.4,13.000000
1,7.2,19.000000
2,22.5,66.000000
3,1.1,29.333333


In [8]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]})

# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=False)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

/var/folders/x9/w1kvj6ms0p52l8j6crt19vmh0000gn/T/ipykernel_16527/727811685.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = np.where(X[col].isnull(),


,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [9]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]})
# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=True)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [10]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]}))
np_toy_train

array([[ 8.3, 23. ],
       [ nan, 29. ],
       [10.2, 36. ],
       [ 3.1,  nan]])

In [11]:
# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame(
    {'Balance': [10.4, np.NaN, 22.5, 1.1], 
     'Age': [13, 19, 66, np.NaN]}))
np_toy_test

array([[10.4, 13. ],
       [ nan, 19. ],
       [22.5, 66. ],
       [ 1.1,  nan]])

In [12]:
# обучаем модель
imp.fit(np_toy_train)
# выполняем преобразование игрушечного
# обучающего массива NumPy
np_toy_train = imp.transform(np_toy_train)
np_toy_train

array([[ 8.3       , 23.        ],
       [ 7.2       , 29.        ],
       [10.2       , 36.        ],
       [ 3.1       , 29.33333333]])

In [13]:
# выполняем преобразование игрушечного
# тестового массива NumPy
np_toy_test = imp.transform(np_toy_test)
np_toy_test

array([[10.4       , 13.        ],
       [ 7.2       , 19.        ],
       [22.5       , 66.        ],
       [ 1.1       , 29.33333333]])

In [14]:
# создаем 1-D массив NumPy
array = np.array([8.3, np.NaN, 10.2, 3.1])
array

array([ 8.3,  nan, 10.2,  3.1])

In [15]:
# проверяем размерность массива
array.ndim

1

In [16]:
# проверяем работу класса
imp.fit(array)
array = imp.transform(array)
array

array([[ 8.3],
       [ 7.2],
       [10.2],
       [ 3.1]])

In [17]:
# проверяем размерность массива
array.ndim

2

In [18]:
# создаем игрушечный датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]})

# создаем пустой словарь encoder_dict
encoder_dict = {}

# по каждой переменной
for col in toy_train.columns:
    # печатаем имя
    print(col)
    # вычисляем среднее и записываем в словарь
    encoder_dict[col] = toy_train[col].mean()
    # печатаем словарь
    print(encoder_dict)
    print('')
    
# печатаем итоговый словарь
print('итоговый словарь', encoder_dict)

Balance
{'Balance': 7.2}

Age
{'Balance': 7.2, 'Age': 29.333333333333332}

итоговый словарь {'Balance': 7.2, 'Age': 29.333333333333332}


In [19]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame(
    {'Balance': [8.3, np.NaN, 10.2, 3.1], 
     'Age': [23, 29, 36, np.NaN]}))

# создаем пустой словарь encoder_dict
encoder_dict = {}

# по каждой переменной
for col in range(np_toy_train.shape[1]):
    # печатаем имя
    print(col)
    # вычисляем среднее и записываем в словарь
    encoder_dict[col] = np.nanmean(np_toy_train[:, col])
    # печатаем словарь
    print(encoder_dict)
    print('')
    
# печатаем итоговый словарь
print('итоговый словарь', encoder_dict)  

0
{0: 7.2}

1
{0: 7.2, 1: 29.333333333333332}

итоговый словарь {0: 7.2, 1: 29.333333333333332}


In [20]:
# пишем класс KNN-модели
class KNN_Estimator():
    """ 
    KNN-модель.
    
    Параметры:
    -----------
    k: int, по умолчанию 2
        Количество ближайших соседей, которое определяет
        класс/значение предсказываемого наблюдения.
    task: string, 'classification' по умолчанию
        Тип решаемой задачи.   
    """
    
    # пишем защищенный метод, вычисляющий евклидово расстояние
    def _euclidean_distance(self, x1, x2):
        """ 
        Вычисляет евклидово расстояние между двумя векторами. 
        """
        distance = 0
        for i in range(len(x1)):
            distance += pow((x1[i] - x2[i]), 2)
        return math.sqrt(distance)
    
    # пишем защищенный метод голосования
    def _vote(self, neighbor_labels):
        """ 
        Возвращает самый часто встречающийся класс 
        среди ближайших соседей.
        """
        # подсчитываем абсолютные частоты классов
        # для каждого наблюдения
        counts = np.bincount(neighbor_labels.astype('int'))
        # возвращаем индекс максимального значения -
        # максимальной абсолютной частоты
        return counts.argmax()

    def __init__(self, k=5, task='classification'):
        # инициализируем k - количество ближайших соседей
        self.k = k
        # решаемая задача
        self.task = task
        # создаем пустой список, в котором будем
        # хранить ближайших соседей для
        # каждого наблюдения набора
        self.k_nearest_neighbors_ = []

    def fit(self, X, y):
        # просто запоминаем обучающий массив признаков
        # и обучающий массив меток
        self.X_memorized = X
        self.y_memorized = y

    def predict(self, X):
        # создаем массив прогнозов, равный
        # длине тестового набора
        y_pred = np.empty(X.shape[0])
        # для каждого наблюдения тестового набора
        # предсказываем наиболее часто встречающийся
        # класс/среднее значение среди k ближайших соседей
        if self.task == 'classification':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = self._vote(self.k_nearest_neighbors_[i])
            
        if self.task == 'regression':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = np.mean(self.k_nearest_neighbors_[i])
                
        return y_pred

In [21]:
# создаем обучающий массив признаков
X_trn = np.array([[0.1, 0.2, 0.3], 
                    [0.7, 0.5, 0.2],
                  [0.1, 0.2, 0.2],
                  [0.9, 0.7, 3.5],
                  [0.2, 0.4, 1.4],
                  [0.4, 0.1, 0.5]])

# создаем обучающий массив меток для классификации
y_trn = np.array([1, 0, 1, 0, 0, 1])

# создаем тестовый массив признаков
X_tst = np.array([[0.1, 0.7, 1.1], 
                  [0.5, 0.3, 2.8],
                  [0.1, 0.1, 0.2],
                  [0.9, 0.7, 1.5]])

In [22]:
# обучаем модель KNN-классификации
knn = KNN_Estimator(k=3, task='classification')
knn.fit(X_trn, y_trn)

In [23]:
# получаем прогнозы для тестового 
# массива признаков
pred = knn.predict(X_tst)
pred

array([1., 0., 1., 0.])

In [24]:
# посмотрим ближайших соседей по каждому 
# наблюдению тестового массива признаков
knn.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]

In [25]:
# создаем обучающий массив меток для регрессии
y_trn = np.array([1.2, 0.5, 1.4, 2.2, 3.5, 5.9])

In [26]:
# обучаем модель KNN-регрессии
knn = KNN_Estimator(k=3, task='regression')
knn.fit(X_trn, y_trn)

In [27]:
# получаем прогнозы для тестового 
# массива признаков
pred = knn.predict(X_tst)
pred

array([3.53333333, 3.86666667, 2.83333333, 3.3       ])

In [28]:
# посмотрим ближайших соседей по каждому 
# наблюдению тестового массива признаков
knn.k_nearest_neighbors_

[array([3.5, 5.9, 1.2]),
 array([2.2, 3.5, 5.9]),
 array([1.4, 1.2, 5.9]),
 array([3.5, 5.9, 0.5])]